In [ ]:
from nltk.corpus import brown
print(brown.tagged_words())
# for i in brown.tagged_words():
#     print(i)

# Preprocessing of data

In [260]:
# --------------------------------------Vocabulary of words------------------------------------------------
def numeric(stri):
    for i in range(len(stri)):
        if stri[i].isdigit():
            return False
    return True

def PreprocessData(data):
    temp = []
    for i in data:
        if i[0] != i[1] and not i[0].isdigit() and numeric(i[0]):
            if '-' in i[1]:
                ll = i[1].split('-')
                brownTag = ll[0]
            elif '+' in i[1]:
                ll = i[1].split('+')
                brownTag = ll[0]
#                 brownTag = i[1][:i[1].index('+')]
            else:
                brownTag = i[1]
            
            brownTag = brownTag.replace('*','')
            brownTag = brownTag.replace('$','')
            print((i[0],brownTag))
            temp.append((i[0],brownTag))
        elif i[0] == '.':
            temp.append(i)
        else:
            pass
    return temp

# ----------------------------------- Sentences ----------------------------------------------------------

def form_sentences(pdata):
    global sentences
    sent = []
    sent.append(tuple(start_sentence1))
    sent.append(tuple(start_sentence2))
    for i in pdata:
        vocab.add(i[0])
        if i[0] != i[1]:
            sent.append(i)
        else:
            sent.append(tuple(end_sentence))
            sentences.append(sent)
            sent = []
            sent.append(tuple(start_sentence1))
            sent.append(tuple(start_sentence2))



# Populating data structure

In [261]:
def formTagDictionary(pdata):
    global tags_dict
    for p in pdata:
        tags_dict[start_sentence1[1]] = tags_dict.get(start_sentence1[1],0) + 1
        tags_dict[start_sentence2[1]] = tags_dict.get(start_sentence2[1],0) + 1
        tags_dict[end_sentence[1]] = tags_dict.get(end_sentence[1],0) + 1
        for i in p:
            tags_dict[i[1]] = tags_dict.get(i[1],0) + 1
        
def formTagWordPairDict(pdata):
    global word_tag_pair
    for p in pdata:
        for i in p:
            word_tag_pair[(i[1],i[0])] = word_tag_pair.get((i[1],i[0]),0) + 1
        
def formTagBigrams():
    global training_data
    for s in training_data:
        l = len(s)-2
        for i in range(l):
            tag_bigram[(s[i][1],s[i+1][1])] = tag_bigram.get((s[i][1],s[i+1][1]),0) + 1
        
def formTagTrigrams():
    global training_data
    for s in training_data:
        l = len(s)-3
        for i in range(l):
            tag_trigram[(s[i][1],s[i+1][1],s[i+2][1])]=tag_trigram.get((s[i][1],s[i+1][1],s[i+2][1]),0)+1

# Calculating transition and emission probabilities

In [262]:
# ------------------------------ Calculating transition Probabilities ------------------------------------
l1 = 0.1
kk = 0.1
l2 = 0
def transition_probability(tri):
    global l1,l2,kk
    wn_0 = tri[0]
    wn_1 = tri[1]
    wn_2 = tri[2]
    x1 = l1*1.0*(tag_trigram.get( (wn_0, wn_1, wn_2) , 0) + kk)/(tag_bigram.get( (wn_0, wn_1) , 0) + kk*len(tags_dict.keys()) )
    x2 = l2*1.0*(tag_bigram.get( (wn_1, wn_2) , 0) + kk)/(tags_dict.get(wn_1, 0) + kk*len(tags_dict.keys()) )
    return x1+x2

# ------------------------------ Calculating emission Probabilities ------------------------------------
def emission_probability(bi):
    
    global  word_tag_pair
    global l1,l2,kk
    wn_0 = bi[0]
    wn_1 = bi[1]
    x1 = l1*1.0*(word_tag_pair.get( (wn_0, wn_1) , 0) + kk)/(tags_dict.get(wn_0 , 0) + kk*len(tags_dict) )
    return x1
#     for k,v in word_tag_pair.items():
#         num = v
#         deno = tags_dict.get(k[0],0)
#         if deno != 0:
#             emission_probs[k] = float(num)/float(deno)
#         else:
#             emission_probs[k] = float(num+1)/float(deno+1.0*len(word_tag_pair))
    

# Finding the tag sequence using Viterbi Algorithm

In [263]:
# --------------------------- VITERBI ALGORITHM ------------------------------------------------------

def viterbi(sentence):
    dp = []
    tag_seq = []
    #INITIALIZE DATA STRUCT
    for i in range(len(sentence)):
        dp.append([])
        tag_seq.append([])
        for j in range(len(tags_dict)):
            dp[i].append([])
            tag_seq[i].append([])
            for each in TAG_SET:
                dp[i][j].append(0)
                tag_seq[i][j].append(0)

    for i in range(len(TAG_SET)):
        tag_gram = ('<ssos1>', '<ssos2>', TAG_SET[i])
        trans = transition_probability(tag_gram)
        word_gram = ('<ssos2>', '<sos2>')
        emit = emission_probability(word_gram)
        dp[0][1][i] = trans*emit

    for p in range(1, len(sentence)):
        for k in range(len(TAG_SET)):
            for j in range(len(TAG_SET)):
                maximum = -1
                word_gram = (TAG_SET[k], sentence[p][0])
                emit = emission_probability(word_gram)
                for i in range(len(TAG_SET)):
                    tag_gram = (TAG_SET[i], TAG_SET[j], TAG_SET[k])
                    trans = transition_probability(tag_gram)
                    temp = dp[p-1][i][j] * trans
                    if temp >= maximum:
                        maximum = temp
                        max_pos = i
                dp[p][j][k] = maximum * emit
                tag_seq[p][j][k] = max_pos
    
    final_tags = []
    maximum = -1
    for i in range(len(TAG_SET)):
        for j in range(len(TAG_SET)):
            temp = dp[-1][i][j]
            if temp > maximum:
                maximum = temp
                max_i = i
                max_j = j
                
    final_tags.append(TAG_SET[max_j])
    final_tags.append(TAG_SET[max_i])
    p = len(sentence) - 1
    while p >= 0:
        next = tag_seq[p][max_i][max_j]
        max_j = max_i
        max_i = next
        final_tags.append(TAG_SET[next])
        p -= 1
    final_tags.reverse()
    return final_tags


# MAIN FUNCTION

In [ ]:
# -------------------------------------- MAIN FUNCTION ------------------------------------------------------

import string

sentences = []
vocab = set()

start_sentence1 = ("<sos1>","<ssos1>")
start_sentence2 = ("<sos2>","<ssos2>")
end_sentence = ("<eos>","<eeos>")

raw_data = brown.tagged_words()
processed_data = PreprocessData(raw_data[:400])
form_sentences(processed_data)

training_data = []
testing_data = []

for i in range(len(sentences)):
    if i < 0.8*len(sentences):
        training_data.append(sentences[i])
    else:
        testing_data.append(sentences[i])

tag_trigram = {}
tag_bigram = {}
word_tag_pair = {}
word_word_tag_tri = {}
tags_dict = {}
transition_probs = {}
emission_probs = {}

formTagDictionary(training_data)
formTagWordPairDict(training_data)
formTagTrigrams()
formTagBigrams()
# transition_probability()
# emission_probability()
TAG_SET = list(tags_dict.keys())

print(len(TAG_SET))

# print(TAG_SET)

# for sents in sentences:
#     print(sents)
#     print("\n")

# print(tag_trigram)
# print(tag_bigram)
# print(transition_probs)
# print("\n\n")
# print(emission_probs)


# print(tags_dict)
# print(len(tags_dict))
# print(word_tag_pair)
# print(len(training_data))
mx = (-1, -1, -1)
ans = (-1, -1, -1)
while kk<=1:
    l1 = 0.1
    while l1<=1:
        l2 = 0
        while l2<=1:
            if (l2 + l1) > 1:
                break

            sent_accuracy = 0
            correct_tokens = 0
            total_tokens = 0
            avg = 0
            for sent in testing_data:
                token_accuracy = 0
                result = viterbi(sent)[2:]
                answer = [word[1] for word in sent]
                if result == answer:
                    sent_accuracy += 1
                else:
                    pass
                    print("Predicted", result)
                    print("Actual", answer)
                    
                total_tokens += len(result)
                for i in range(len(result)):
                    if result[i] == answer[i]:
                        correct_tokens += 1
                        token_accuracy += 1
                avg += 1.0*token_accuracy/len(sent)

            R1 = 1.0*sent_accuracy/len(testing_data)
            R2 = 1.0*correct_tokens/total_tokens
            R3 = 1.0*avg/len(testing_data)

            result = (R3, R1, R2)
            print("Accuracy: ",R2*100)
            #print(kk, l1, l2, ':', result)
            if result > mx:
                mx = result
                ans = (kk, l1, l2)
            l2 += 0.1

        l1 += 0.1

    kk += 0.1
#print(ans)
#print(mx)


('The', 'AT')
('Fulton', 'NP')
('County', 'NN')
('Grand', 'JJ')
('Jury', 'NN')
('said', 'VBD')
('Friday', 'NR')
('an', 'AT')
('investigation', 'NN')
('of', 'IN')
("Atlanta's", 'NP')
('recent', 'JJ')
('primary', 'NN')
('election', 'NN')
('produced', 'VBD')
('no', 'AT')
('evidence', 'NN')
('that', 'CS')
('any', 'DTI')
('irregularities', 'NNS')
('took', 'VBD')
('place', 'NN')
('The', 'AT')
('jury', 'NN')
('further', 'RBR')
('said', 'VBD')
('in', 'IN')
('term-end', 'NN')
('presentments', 'NNS')
('that', 'CS')
('the', 'AT')
('City', 'NN')
('Executive', 'JJ')
('Committee', 'NN')
('which', 'WDT')
('had', 'HVD')
('over-all', 'JJ')
('charge', 'NN')
('of', 'IN')
('the', 'AT')
('election', 'NN')
('deserves', 'VBZ')
('the', 'AT')
('praise', 'NN')
('and', 'CC')
('thanks', 'NNS')
('of', 'IN')
('the', 'AT')
('City', 'NN')
('of', 'IN')
('Atlanta', 'NP')
('for', 'IN')
('the', 'AT')
('manner', 'NN')
('in', 'IN')
('which', 'WDT')
('the', 'AT')
('election', 'NN')
('was', 'BEDZ')
('conducted', 'VBN')
('The

Predicted ['AT', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBD', 'BEZ', 'VBG', 'IN', 'HV', 'DTS', 'CD', 'NNS', 'MD', 'BE', 'VBN', 'IN', 'AT', 'JJT', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'NN', 'NN', 'NN', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '<eeos>']
Accuracy:  50.0
Predicted ['<ssos1>', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'BER', 'QL', 'VBN', 'CC', 'VB', 'AT', 'NN', 'RBR', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'IN', 'AT', 'NN', 'VBZ', 'DTS', 'CD', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'AP', 'NN', 'VB', 'VBG', 'NNS', 'CC', 'VB', 'AT', 'JJ', 'NN', 'CS', 'CS', 'AT', 'JJ', 'NN', 'IN', 'AT', 'NN', 'MD', 'BE', 'VBN', '<eeos>']
Predicted ['<ssos1>', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBD', 'BEZ', 'VBG', 'IN', 'HV', 'DTS', 'CD', 'NNS', 'MD', 'BE', 'VBN', 'IN', 'AT', 'JJT', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 

Predicted ['AT', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'BER', 'QL', 'VBN', 'CC', 'VB', 'AT', 'NN', 'RBR', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'IN', 'AT', 'NN', 'VBD', 'PPS', 'DOD', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'AP', 'NN', 'VB', 'VBG', 'NNS', 'CC', 'VB', 'AT', 'JJ', 'NN', 'CS', 'CS', 'AT', 'JJ', 'NN', 'IN', 'AT', 'NN', 'MD', 'BE', 'VBN', '<eeos>']
Predicted ['AT', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBD', 'BEZ', 'VBG', 'IN', 'VBZ', 'DTS', 'CD', 'NNS', 'MD', 'BE', 'VBN', 'IN', 'AT', 'JJT', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'NN', 'NN', 'NN', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '<eeos>']
Accuracy:  50.0
Predicted ['AT', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'BER', 'QL', 'VBN', 'CC', 'VB', 'AT', 'NN', 'RBR', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'IN', 'AT', 'NN', 'VBD', 'PPS', 'DOD', '<eeos>']
Actual ['<ssos1>', '<ss

Predicted ['AT', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBD', 'BEZ', 'VBG', 'IN', 'VBZ', 'DTS', 'CD', 'NNS', 'MD', 'BE', 'VBN', 'IN', 'AT', 'JJT', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'NN', 'NN', 'NN', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'JJ', 'NNS', '<eeos>']
Accuracy:  50.0
Predicted ['AT', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'BER', 'QL', 'VBN', 'CC', 'VB', 'AT', 'NN', 'RBR', 'VBD', 'CS', 'AT', 'NN', 'VBD', 'IN', 'AT', 'NN', 'VBD', 'PPS', 'DOD', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'PPS', 'VBD', 'CS', 'AT', 'AP', 'NN', 'VB', 'VBG', 'NNS', 'CC', 'VB', 'AT', 'JJ', 'NN', 'CS', 'CS', 'AT', 'JJ', 'NN', 'IN', 'AT', 'NN', 'MD', 'BE', 'VBN', '<eeos>']
Predicted ['AT', '<ssos2>', 'AT', 'JJ', 'NN', 'VBD', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBD', 'BEZ', 'VBG', 'IN', 'VBZ', 'DTS', 'CD', 'NNS', 'MD', 'BE', 'VBN', 'IN', 'AT', 'JJT', '<eeos>']
Actual ['<ssos1>', '<ssos2>', 'AT', '